In [1]:
import os
import cv2
import numpy as np
from PIL import Image
import json
import datetime
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import logging

class AutomatedAnnotator:
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.annotations = {}
        self.annotations_file = os.path.join(image_dir, "automated_annotations.json")
        
       
        logging.basicConfig(
            filename='annotation_log.txt',
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        
        
        try:
            self.model = ResNet50(weights='imagenet')
            logging.info("Successfully loaded ResNet50 model")
        except Exception as e:
            logging.error(f"Error loading model: {str(e)}")
            raise
        
        
        if os.path.exists(self.annotations_file):
            with open(self.annotations_file, 'r') as f:
                self.annotations = json.load(f)
                logging.info(f"Loaded {len(self.annotations)} existing annotations")
    
    def detect_features(self, image_path):
        """Detect various features in the image using OpenCV"""
        try:
            
            img = cv2.imread(image_path)
            if img is None:
                raise ValueError(f"Could not read image: {image_path}")
            
            features = {}
            
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            
            edges = cv2.Canny(gray, 100, 200)
            features['edge_density'] = np.mean(edges > 0)
            
            
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            features['num_contours'] = len(contours)
            
            
            colors = cv2.mean(img)
            features['avg_color'] = {
                'blue': colors[0],
                'green': colors[1],
                'red': colors[2]
            }
            
            return features
            
        except Exception as e:
            logging.error(f"Error in detect_features for {image_path}: {str(e)}")
            return None
    
    def classify_image(self, image_path):
        """Classify image using ResNet50"""
        try:
            img = Image.open(image_path)
            img = img.convert('RGB')
            img = img.resize((224, 224))
            x = np.array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            
            preds = self.model.predict(x)
            predictions = decode_predictions(preds, top=3)[0]
            
            return [{'label': label, 'confidence': float(confidence)} 
                   for _, label, confidence in predictions]
            
        except Exception as e:
            logging.error(f"Error in classify_image for {image_path}: {str(e)}")
            return None
    
    def generate_description(self, features, classifications):
        """Generate a text description based on features and classifications"""
        try:
            description = []
            
            
            if classifications:
                top_class = classifications[0]['label'].replace('_', ' ')
                confidence = classifications[0]['confidence']
                description.append(f"This image appears to be a {top_class} ({confidence:.1%} confidence)")
            
            
            if features:
                
                if features['edge_density'] > 0.1:
                    description.append("The image contains significant detail/edges")
                else:
                    description.append("The image is relatively smooth")
                
                
                colors = features['avg_color']
                dominant_color = max(colors.items(), key=lambda x: x[1])[0]
                description.append(f"The dominant color tone is {dominant_color}")
                
                
                if features['num_contours'] > 50:
                    description.append("The image is complex with many distinct shapes")
                else:
                    description.append("The image has a simple composition")
            
            return " ".join(description)
            
        except Exception as e:
            logging.error(f"Error in generate_description: {str(e)}")
            return "Could not generate description"
    
    def process_directory(self):
        """Process all images in the directory"""
        image_files = [f for f in os.listdir(self.image_dir) 
                      if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        total_files = len(image_files)
        logging.info(f"Starting to process {total_files} images")
        
        for i, image_file in enumerate(image_files, 1):
            try:
                image_path = os.path.join(self.image_dir, image_file)
                
                
                if image_file in self.annotations:
                    logging.info(f"Skipping {image_file} - already annotated")
                    continue
                
                logging.info(f"Processing image {i}/{total_files}: {image_file}")
                
                
                features = self.detect_features(image_path)
                
                
                classifications = self.classify_image(image_path)
                
                
                description = self.generate_description(features, classifications)
                
                
                self.annotations[image_file] = {
                    'timestamp': datetime.datetime.now().isoformat(),
                    'features': features,
                    'classifications': classifications,
                    'description': description,
                    'automated': True
                }
                
                
                if i % 10 == 0:
                    self.save_annotations()
                    
                print(f"Processed {i}/{total_files}: {image_file}")
                
            except Exception as e:
                logging.error(f"Error processing {image_file}: {str(e)}")
                continue
        
        
        self.save_annotations()
        logging.info("Finished processing all images")
    
    def save_annotations(self):
        """Save annotations to JSON file"""
        try:
            with open(self.annotations_file, 'w') as f:
                json.dump(self.annotations, f, indent=4)
            logging.info(f"Saved annotations to {self.annotations_file}")
        except Exception as e:
            logging.error(f"Error saving annotations: {str(e)}")


if __name__ == "__main__":
    image_dir = r"C:\Users\eKasi_SWT_COM00862\Downloads\doctor\doctor"
    annotator = AutomatedAnnotator(image_dir)
    annotator.process_directory()

102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
Processed 1/92: 1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Processed 2/92: 100.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Processed 3/92: 103.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
Processed 4/92: 104.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
Processed 5/92: 105.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Processed 6/92: 106.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Processed 7/92: 107.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Processed 8/92: 108.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
Processed 9/92: 126.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
Processed 10/92: 13.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Processed 11/92: 1a5b485681d0a59ce6c4faf95610dcca.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Processed 12/92: 21.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Processed 13/92: 23.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Processed 14/92: 28.jpg